In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [4]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7fd58e3e1310>)

In [6]:
naver_df = pd.read_table('ratings.txt')
naver_df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [11]:
#리뷰 갯수 출력
print("리뷰 개수 : ",len(naver_df))

리뷰 개수 :  200000


In [12]:
# NuLL확인
naver_df.isnull().values.any()

True

In [13]:
# NULL 행 제거 
naver_df = naver_df.dropna(how = 'any')
#NULL존재 확인
naver_df.isnull().values.any()

False

In [14]:
#리뷰 갯수 출력하기
print("리뷰 개수 : ",len(naver_df))

리뷰 개수 :  199992


In [15]:
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [17]:
corpus = 'naver_review.txt'
prefix = 'naver'
vocab_size = 5000

spm.SentencePieceTrainer.Train(f'--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size}' +
                               ' --model_type=bpe' + 
                               ' --max_sentence_length=9999')
                               

In [19]:
# vocab불러오기 pd.read_csv
vocab_list = pd.read_csv('naver.vocab', sep = '\t', header=None, quoting=csv.QUOTE_NONE)
# vocab 10개출력
vocab_list.sample(10)

,0,1
4257,ᅩ,-4254
4279,젼,-4276
257,▁또,-254
1476,▁자기,-1473
1147,▁영화중,-1144
98,▁유,-95
3849,숨,-3846
2171,▁류,-2168
1497,이게,-1494
959,▁OOO,-956


In [20]:
# vocab_file load하기
sp = spm.SentencePieceProcessor()
vocab_file = 'naver.model'
sp.load(vocab_file)

True

In [21]:
lines = [
  "뭐 이딴 것도 영화냐.",
  "진짜 최고의 영화입니다 ㅋㅋ",
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

뭐 이딴 것도 영화냐.
['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']
[132, 966, 1296, 2590, 3276]

진짜 최고의 영화입니다 ㅋㅋ
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]



In [23]:
sp.GetPieceSize()

5000

In [24]:
sp.IdToPiece(123)

'▁배우'

In [28]:
sp.PieceToId('▁배우') 

123

In [34]:
sp.DecodeIds(sp.encode_as_ids(lines[0]))

'뭐 이딴 것도 영화냐.'

In [36]:
sp.DecodePieces(sp.encode_as_pieces(lines[0])) # 서브워드 시퀀스 입력

'뭐 이딴 것도 영화냐.'

In [37]:
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=str))
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=int))

['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]


# word Embedding

## One-Hot Encoding 실습

In [38]:
import numpy as np

#입력문장
raw_inputs = [
             '나는 학생입니다.',
             '나는 좋은 선생님 입니다.',
             '당신은 매우 좋은 선생님 입니다.'
]

raw_labels = [1,0,0] # 학생이 존재하면 1 없으면 0

In [41]:
#문장을 띄어쓰기 단위로 분할

words = []
for s in raw_inputs:
  words.extend(s.split(' '))

#중복단어 제거
words = list(dict.fromkeys(words))

word_to_idx = {
    "[PAD]" : 0,
    "[UNK]" : 1,
}

for w in words:
  word_to_idx[w] = len(word_to_idx)

idx_to_word = {i : w for w, i in word_to_idx.items()}

print(idx_to_word)


{0: '[PAD]', 1: '[UNK]', 2: '나는', 3: '학생입니다.', 4: '좋은', 5: '선생님', 6: '입니다.', 7: '당신은', 8: '매우'}


In [52]:
word_to_idx

{'[PAD]': 0,
 '[UNK]': 1,
 '나는': 2,
 '당신은': 7,
 '매우': 8,
 '선생님': 5,
 '입니다.': 6,
 '좋은': 4,
 '학생입니다.': 3}

In [42]:
train_inputs = []
for s in raw_inputs:
  row = [word_to_idx[w] for w in s.split()]
  row += [0] * (5 - len(row))
  train_inputs .append(row)

train_inputs = np.array(train_inputs)

print(train_inputs)

[[2 3 0 0 0]
 [2 4 5 6 0]
 [7 8 4 5 6]]


In [45]:
# one -hot metrix 생성
onehot_metrix = np.eye(len(word_to_idx))
print(onehot_metrix)

[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [46]:
train_onehots=  onehot_metrix[train_inputs]
print(train_onehots) # 메모리 낭비가 심하다.

[[[0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0.]]]


In [48]:
print(np.argmax(train_onehots, axis = -1))

[[2 3 0 0 0]
 [2 4 5 6 0]
 [7 8 4 5 6]]


In [49]:
x = np.argmax(train_onehots, axis = -1)

In [50]:
import tensorflow as tf
import tensorflow.keras.layers as L

In [51]:
x_len = train_onehots.shape
inp = tf.convert_to_tensor(x, dtype = tf.int32)
inp_len = tf.convert_to_tensor(x_len, dtype = tf.int32)

print(inp)
print(inp_len)

tf.Tensor(
[[2 3 0 0 0]
 [2 4 5 6 0]
 [7 8 4 5 6]], shape=(3, 5), dtype=int32)
tf.Tensor([3 5 9], shape=(3,), dtype=int32)


In [53]:
vocab = 1000
dim = 3
embed = L.Embedding(vocab, dim)

In [54]:
embed(inp)

<tf.Tensor: shape=(3, 5, 3), dtype=float32, numpy=
array([[[ 1.9377578e-02,  4.4603262e-02, -3.3578716e-02],
        [ 7.4099749e-05,  3.5429273e-02, -3.7045367e-03],
        [-2.1648562e-02, -4.7007062e-02, -3.0328631e-03],
        [-2.1648562e-02, -4.7007062e-02, -3.0328631e-03],
        [-2.1648562e-02, -4.7007062e-02, -3.0328631e-03]],

       [[ 1.9377578e-02,  4.4603262e-02, -3.3578716e-02],
        [ 7.7015534e-03,  1.4470223e-02,  2.0395145e-03],
        [ 4.9058344e-02,  2.1654677e-02, -8.7254122e-04],
        [-3.8616288e-02, -8.9075677e-03, -4.4052314e-02],
        [-2.1648562e-02, -4.7007062e-02, -3.0328631e-03]],

       [[-3.1596266e-02,  2.2611666e-02, -1.6120147e-02],
        [ 3.8933549e-02, -4.4198420e-02, -1.8555950e-02],
        [ 7.7015534e-03,  1.4470223e-02,  2.0395145e-03],
        [ 4.9058344e-02,  2.1654677e-02, -8.7254122e-04],
        [-3.8616288e-02, -8.9075677e-03, -4.4052314e-02]]], dtype=float32)>

# WordNet

-Thesaurus

단어의 유사도를 거리로 표현 

$ similarity(w,w') = - log distance(w,w')$  
  
  
### Local Representation

- 해당 단어 그 자체만 보고 특정값을 맵핑하여 단어를 표현
### distributed Representation
- 그 단어를 표현하고자 주변을 참고하여 단어를 표현

#### beg of words
- 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법
>만드는 과정
>> 1. 각 단어에 고유한 정수 인덱스를 부여한다.
>> 1. 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만든다

##### 문서 단어 행렬 (Document_Term Matrix, DTM)
- 각 문서에서 등장한 단어의 빈도를 행렬의 값으로 표기
- 문서 단어 행렬은 문서들을 서로 비교할 수 있도록 수치화 할 수 있다는 점에 의의!
- 단점
> 희소 표현  
>> 각 문서 벡터의 차원은 원-핫 벡터와 마찬가지로 전체 단어 집합의 크기를 가짐 
>> 많은 문서벡터가 대부분의 0의 값을 가질 수 있음
>> 희소 벡터는 많은 양의 저장공간과 계산을 위한 리소스가 필요  

  > 단순 빈도 수 기반 접근

##### TF-IDF
1. 텍스트 마이닝(Text Mining)에서 중요하게 사용
1. 어떤 단어 w가 문서 d 내에서 얼마나 중요한지 나타내는 수치
1. TF(Tern Frequency)
  - 단어의 문서 내에 출현한 횟수
  - 숫자가 클수록 문서 내에서 중요한 단어
  - 하지만 'the'와 같은 단어도 `TF`값이 매우 클 것

##### IDF(Inverse Document Frequency)
- 그 단어가 출현한 문서의 역수(Inverse)
- 값이 클 수록 'the'와 가이 일반적으로 많이 쓰이는 단어


  

# Beg of words
- 단어의 등장 순서를 고려하지 않은 빈도수 기반의 단어 표현 방법
  1. 각 단어의 고유한 정수 인덱스를 부여
  1. 각 인덱스 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만든다. 

``` python
doc1= 'Jone likes to watch movies. Mary likes movies too'
Bow1 = { "Jone" : 1, "likes" : 2 , "watch" : 1, "movies": 2, "Mary" : 1, "too: :1}
```

In [55]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 7.1 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
     |████████████████████████████████| 448 kB 42.9 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [56]:
from konlpy.tag import Okt
import re

okt = Okt()

In [58]:
# 정규표현식을 통해 온점을 제거하는 정제 작업
token = re.sub("(\.)","","소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.")

token = okt.morphs(token)
token

['소비자', '는', '주로', '소비', '하는', '상품', '을', '기준', '으로', '물가상승률', '을', '느낀다']

In [61]:
word2index = {}
bow = []

for voca in token:
  if voca not in word2index.keys():
    word2index[voca] = len(word2index) # token 을 읽으면서 , 없는 단어를 새로 추가, 
    bow.insert(len(word2index)-1, 1) # 기본값 1을 넣어둔다.
  else:
    index = word2index.get(voca) # 재등장 단어 
    bow[index] = bow[index] + 1

print(word2index)
print(bow)

{'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


### Tensorflow keras tokenizer를 사용해서 bow구하기


In [62]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentence = [
            'Jone likes to watch movies. \
            Mary likes movies too! \
            Mary also likes to watch football games.']

In [68]:
def print_bow(sentence):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(sentence) # 단어장 생성
  bow = dict(tokenizer.word_counts) #bow 저장
  print('Bag of words : ', bow)
  print('단어장(Vocabulary)의 크기 : ',len(tokenizer.word_counts))


In [69]:
print_bow(sentence)

Bag of words :  {'jone': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
단어장(Vocabulary)의 크기 :  10


### sklearn의 CountVectorizer을 활용한 bow

In [71]:
from sklearn.feature_extraction.text import CountVectorizer

sentence = ['Jone likes to watch movies. \
            Mary likes movies too! \
            Mary also likes to watch football games.']

vector = CountVectorizer()
print("Bag of Words : ",vector.fit_transform(sentence).toarray()) 
print("각 단어의 인덱스 : ", vector.vocabulary_)

Bag of Words :  [[1 1 1 1 3 2 2 2 1 2]]
각 단어의 인덱스 :  {'jone': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


### 사용자가 직접 정의한 불용어(The an ...)를 제거한 bow 만들기


In [75]:
from sklearn.feature_extraction.text import CountVectorizer

text = ["Family is not an important thing. It's everything"]

vect = CountVectorizer(stop_words = ['the','a','an','is','not'])  # stop_words에 list형식으로 불용어 지정

print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


### CountVectorizer에서 제공하는 자체 불용어를 사용

In [78]:
from sklearn.feature_extraction.text import CountVectorizer

text = ["Family is not an important thing. It's everything"]

vect = CountVectorizer(stop_words = 'english')  #  countervectorizer에서 제공하는 불용어를 제거 (영어 english)

print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1]]
{'family': 0, 'important': 1, 'thing': 2}


### NLTK에서 지원하는 불용어 사용

In [80]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [81]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

text = ["Family is not an important thing. It's everything"]

sw = stopwords.words('english')
vect = CountVectorizer(stop_words = sw)  #  countervectorizer에서 제공하는 불용어를 제거 (영어 english)

print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


# DTM(Document Term Metrix)

- 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것
- 다수의 문서에 대해서 Bow를 하나의 행렬로 표현하고 부르는 용어
--------------------------------------

문서 1 : I like dog  
문서 2 : I like cat  
문서 3 : I like cat I like cat  

In [82]:
import pandas as pd
content = [[0, 1, 1, 1], [1, 0, 1, 1], [2, 0, 2, 2,]]
df = pd.DataFrame(content)
df.index = ['(문서1) I like dog', '(문서2) I like cat', '(문서3) I like cat I like cat']
df.columns = ['cat', 'dog', 'I', 'like']
df

,cat,dog,I,like
(문서1) I like dog,0,1,1,1
(문서2) I like cat,1,0,1,1
(문서3) I like cat I like cat,2,0,2,2


In [89]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

#  get cos similarlity 
def cos_sim(a,b):  # 코사인 유사도는 0~1사이의 값을 가지고, 1에 가까울 수록 유사도가 높다고 판단
  return dot(a,b)/(norm(a) * norm(b))

In [90]:
print(cos_sim(doc1,doc2))
print(cos_sim(doc1,doc3))
print(cos_sim(doc2,doc3))

0.6666666666666667
0.6666666666666667
1.0000000000000002


### sklearn CountVectorizer를 활용한 DTM 구현

In [91]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['Jone likes to watch movies. \
            Mary likes movies too! \
            Mary also likes to watch football games.']

vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

[[1 1 1 1 3 2 2 2 1 2]]
{'jone': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


#### TF-IDF(Term-Frequency-Inverse Document Frequency)
- 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단
- 특정 문서에서만 자주 등장하는 단어는 중요도가 높고 판단

$ TF-IDF(w,d) = \frac{TF(w,d)}{DF(w)}$

In [103]:
from math import log
import pandas as pd
import sys

docs = [
        'John likes to watch movies and Mary likes movies too',
        'James likes to watch TV',
        'Mary also likes to watch football games',
]

In [109]:
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

print('단어장의 크기 :', len(vocab))
print(vocab)

단어장의 크기 : 13
['James', 'John', 'Mary', 'TV', 'also', 'and', 'football', 'games', 'likes', 'movies', 'to', 'too', 'watch']


In [110]:
N =len(docs)

$ idf(d,t) = log\frac{N}{1+df(t)} $

In [114]:
def tf(t,d): # 특정 문서 d에서의 특정 단어 t의 등장 횟수
  return d.count(t)

def idf(t): # 반비례 하는 수
  df = 0 # 특정 단어 t가 등장한 문서의 수
  for doc in docs:
    df += t in doc
  return log(N/(df+1)) + 1

def tfidf(t,d):
  return tf(t,d) * idf(t)

In [115]:
result = []

for i in range(N):
  result.append([])
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]

    result[-1].append(tf(t,d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_.head()

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0,1,1,0,0,1,0,0,2,2,2,1,1
1,1,0,0,1,0,0,0,0,1,0,1,0,1
2,0,0,1,0,1,0,1,1,1,0,1,0,1


In [116]:
result = [] 
for j in range(len(vocab)):
  t = vocab[j]
  result.append(idf(t))

idf_ = pd.DataFrame(result,index = vocab, columns=['idf'])
idf_

,idf
James,1.405465
John,1.405465
Mary,1.000000
TV,1.405465
also,1.405465
and,1.405465
football,1.405465
games,1.405465
likes,0.712318
movies,1.405465


In [117]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))
tfidf_=pd.DataFrame(result, columns=vocab)
tfidf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0.000000,1.405465,1.0,0.000000,0.000000,1.405465,0.000000,0.000000,1.424636,2.81093,1.424636,1.405465,0.712318
1,1.405465,0.000000,0.0,1.405465,0.000000,0.000000,0.000000,0.000000,0.712318,0.00000,0.712318,0.000000,0.712318
2,0.000000,0.000000,1.0,0.000000,1.405465,0.000000,1.405465,1.405465,0.712318,0.00000,0.712318,0.000000,0.712318
